<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


<a href="https://blog.cloudcommander.net" target="_parent"><img src="https://raw.githubusercontent.com/cloud-commander/hexoblog/master/cloud.png" alt="Visit my Blog">
</a>
<br> 
# <span style="font-family:Didot; font-size:3em;"> Cloud Commander </span>


<a href="https://colab.research.google.com/github/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Part2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"></a>
&nbsp;&nbsp;&nbsp;&nbsp;
<a href="https://github.com/cloud-commander/face-mask-detection/blob/master/2_Prepare_Data_Part2.ipynb" target="_parent"><img src="https://img.shields.io/static/v1?logo=GitHub&label=&color=333333&style=flat&message=View%20on%20GitHub" alt="View in GitHub"></a>


# Automatic Face Image Annotation & Masked Photo Generation ##

In the previous notebook, we prepared the unmasked photos in the dataset. In this notebook we will look at pre-processing the images for our masked photo dataset.

Firstly we will identify the bounding boxes for the faces as with the unmasked photos but additionally we will then also place a series of masked into those faces thus making them masked photos.

## Import required libraries

In [1]:
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
from constants import * 
!wget {ANNOTATE}
!wget {PLACE_MASKS}
!wget {MOVE_FILES}

!pip install face_recognition
!pip install wget

from annotate import *
from place_masks import * 
from move_files import * 

import os
import glob
import tarfile

import sys
import random

--2020-06-16 18:30:57--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/config/constants.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 6063 (5.9K) [text/plain]
Saving to: ‘constants.py’

constants.py        100%[===================>]   5.92K  --.-KB/s    in 0s      

2020-06-16 18:30:58 (40.3 MB/s) - ‘constants.py’ saved [6063/6063]

--2020-06-16 18:30:59--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/utils/annotate.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2

## Prepare dataset ###

### Connect to Google Drive


In [2]:
from google.colab import drive

drive.mount('/content/drive/')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive/


### Download faces dataset

In [3]:
%cd {DATASET_DIR_TMP1}
!wget https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/100_faces_320x320_dataset_set_2.zip
!unzip -j 100_faces_320x320_dataset_set_2.zip '*.jpg'
!rm 100_faces_320x320_dataset_set_2.zip

/content/Face-Mask-Detection/dataset/tmp1
--2020-06-16 18:32:10--  https://raw.githubusercontent.com/cloud-commander/face-mask-detection/master/data/100_faces_320x320_dataset_set_2.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3307366 (3.2M) [application/zip]
Saving to: ‘100_faces_320x320_dataset_set_2.zip’

100_faces_320x320_d 100%[===================>]   3.15M  15.6MB/s    in 0.2s    

2020-06-16 18:32:12 (15.6 MB/s) - ‘100_faces_320x320_dataset_set_2.zip’ saved [3307366/3307366]

Archive:  100_faces_320x320_dataset_set_2.zip
  inflating: 07UJWYHLCD.jpg          
  inflating: 0VYZK4SWM5.jpg          
  inflating: 0W6R2QIU5A.jpg          
  inflating: 0WF57M1IBJ.jpg          
  inflating: 0WXKO4TSI6.jpg          
  inflating: 19309Q6KY4.jpg          
  inf

## Annotate the images
Calls the annotate function to automatically annotate the faces in our image folder and outputs XML files

In [4]:
annotate(DATASET_DIR_TMP1,DATASET_DIR_TMP2,"MASKED")

## Generate Masked Photos

### Download required dependency and extract to folder

In [5]:
os.chdir(DATASET_DIR_UNPREP)
tf = tarfile.open(wget.download(MASKS_DATASET))
tf.extractall()

### Cycle through photos and place mask images

In [6]:
model = "cnn" # Select facial recognition model; Available options areh 'hog' or 'cnn'. Hog is faster, cnn is more accurate.

input_folder_list = [os.path.join(DATASET_DIR_TMP1, f) for f in os.listdir(DATASET_DIR_TMP1) if os.path.isfile(os.path.join(DATASET_DIR_TMP1, f))]
mask_folder_list = [os.path.join(DATASET_DIR_UNPREP_MASKS, f) for f in os.listdir(DATASET_DIR_UNPREP_MASKS) if os.path.isfile(os.path.join(DATASET_DIR_UNPREP_MASKS, f))]

for i in range(len(input_folder_list)):
     # get a random mask from from the mask folder
     random_mask_file = random.choice(list(mask_folder_list))
        
     if random_mask_file.endswith(('.png', '.jpg', '.jpeg')): 
      # get the next image from the image folder
      current_image_file = input_folder_list[i]
      if current_image_file.endswith(('.png', '.jpg', '.jpeg')):
        create_mask(current_image_file, random_mask_file, model)


Save to /content/Face-Mask-Detection/dataset/tmp1/2I9MFY4KPJ.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/U8UPYXV8P7.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/NYU4CF7ZZW.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/IVOUZ7AO71.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/P9K1JRRLNF.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/1Y6HSX6TJ1.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/Y0YN36DVLT.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/5EHDHMB62U.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/6UGMKTSLSW.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/J9KV0D4ZLL.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/53MFUERP5Y.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/W2872BT4OT.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/I2GN0G9Q27.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/Z84C1DEY9V.jpg
Save to /content/Face-Mask-Detection/dataset/tmp1/0VYZK4SWM5.jpg
Save to /content/Face-Mas

## Save

### Save the generated images and archive them

In [7]:
#Return a list of items that have both images and XML files
matches = compare_intersect(DATASET_DIR_TMP1,DATASET_DIR_TMP2)

#Move the images and the annotations from the temp location
move_files(DATASET_DIR_TMP1,DATASET_DIR_UNPREP_IMG, matches)
move_files(DATASET_DIR_TMP2,DATASET_DIR_UNPREP_ANNO, matches)

Files moved: 100
Files moved: 100


### Compress Dataset and Save Dataset to Google Drive

Follow this step if you wish to archive your dataset for future use

In [8]:
!zip -r part2-datasetv1.zip {DATASET_DIR_UNPREP}
!gsutil cp part2-datasetv1.zip {DRIVE_DEV}

  adding: content/Face-Mask-Detection/dataset/unprepared/ (stored 0%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/ (stored 0%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/2I9MFY4KPJ.jpg (deflated 1%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/U8UPYXV8P7.jpg (deflated 1%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/NYU4CF7ZZW.jpg (deflated 1%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/IVOUZ7AO71.jpg (deflated 1%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/P9K1JRRLNF.jpg (deflated 1%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/1Y6HSX6TJ1.jpg (deflated 1%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/Y0YN36DVLT.jpg (deflated 1%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/5EHDHMB62U.jpg (deflated 1%)
  adding: content/Face-Mask-Detection/dataset/unprepared/images/6UGMKTSLSW.jpg (deflated 1%)
  adding: conten